# Import Libraries and Packages

In [1]:
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [2]:
# import necessary libraries
from datetime import date, timedelta
import re
from bs4 import BeautifulSoup
import requests
import urllib
import re
import numpy as np
import pandas as pd
import threading
from threading import Timer
from time import sleep
import time

# Read CSV and TXT Files

In [3]:
# bad_teams = open(r"C:\Users\James Seelos\Documents\Personal Projects\CBB Predictor\data\bad_teams.txt", "r").read().strip().split("\n")

In [4]:
matchups_df = pd.read_csv(r"C:\Users\James Seelos\Documents\Personal Projects\CBB Predictor\data\cbb_matchups.csv")
matchups_dict = matchups_df.to_dict('records')

### Functions for URLS

In [5]:
def getHTMLdocument(url):
    """ 
    Sends GET request to the givin url and returns the html document 
 
    Args: 
        url - string: HTTP URL to send the GET request
    Returns: 
        Response of GET request in HTML format
     
    """ 
    response = requests.get(url)

    return response.text

In [6]:
# function to test if a url returns a 404
def testURL(url):
    """ 
        Sends GET request to the givin url and returns the html document 
 
    Args: 
        url - string: HTTP URL to send the GET request
    Returns: 
        Response code of GET request along with the url
    """ 
    response_code = requests.get(url).status_code
    response_string = f'URL: {url}\nResponse: {response_code}\n'
    
    return response_string

# Create Matchups CSV

### Functions

In [7]:
def createDates(start, end):
    """ 
    Creates dates for a specified range 
 
    Args: 
        start - date: starting date
        end - date: ending date 
    Returns: 
        date_list - list<datetime.date>: list of dates from start -> end 
 
    """
    date_list = []
    delta = end - start 

    for i in range(delta.days + 1):
        day = start + timedelta(days=i)
        if day.month > 10 or day.month < 5: # Only need months 11 - 4 (November - April)
            date_list.append(day)
    
    return date_list

In [8]:
# function to create urls to use to get matchups
def createDatesURL(date_list):
    """ 
    Formats and creates urls for the dates that are given
 
    Args: 
        date_list - list<datetime.date>: list of dates 
    Returns: 
        date_url_list - list<str>: list of urls for the given dates 
        
    """
    date_url_list = []

    for date in date_list:
        
        # Format the dates into strings for the url
        year = str(date.year)
        month = str(date.month) if len(str(date.month)) == 2 else "0" + str(date.month)
        day = str(date.day) if len(str(date.day)) == 2 else "0" + str(date.day)
        season = f'{date.year}-{date.year + 1}' if len(str(date.month)) == 2 else f'{date.year - 1}-{date.year}'
        
        url = f'https://newsday.sportsdirectinc.com/sports-scores/College-Basketball-Scores-Matchups.aspx?Year={year}&Period={month}{day}&CurrentSeason={season}'
        date_url_list.append({"url": url, "date": date})
        
    return date_url_list

In [9]:
# function to scrape webpage to get all matchups
def getMatchups(date_url_list):
    """ 
    Scrapes website to gather all games that were played in a given time frame
    Will gather the teams, rankings, url for the box score, and date
 
    Args: 
        date_url_list - list<dict>: dictionary containing all the urls along with the dates 
    Returns: 
        matchups_list - list<dict>: dictionary containing necessary matchup components 
 
    """
    matchups_list = []
    
    for date_url_index in range(len(date_url_list)):
        
        html_document = getHTMLdocument(date_url_list[date_url_index]['url'])
        soup = BeautifulSoup(html_document, 'html.parser') # soup object
        results = soup.find(id='Scoreboard_7_All_Games') # all of the necessary info is in this div 

        if results is not None: # dates that do not have games played on them will be None
            
            matchup_elements = results.find_all('div', class_='sdi-so-title') # each div in matchup elements is a matchup
            
            for matchup_index in range(len(matchup_elements)):
            
                matchup_text = matchup_elements[matchup_index].text.strip()
                teams = re.split(r'( vs )|( at )', matchup_text) # ex. Purdue(1) at Michigan -> ['Purdue(1)', ' at ', 'Michigan']
                teams = [i for i in teams if i is not None]
                neutral = True if teams[1] == ' vs ' else False # ' vs ' indicates a neutral game 

                for team_index in range(0, len(teams), 2):
                    team = re.split(r'\((?=[\d])|(?<=[\d])\)', teams[team_index]) # Extracts ranking from team ex. Purdue(1) -> ['Purdue', '1']
                    team = [i for i in team if i != ""]

                    if team_index == 0: # 0 index is away team
                        away_team = team[0]
                        away_team_ranking = int(team[1]) if len(team) == 2 else 0 # If team has no ranking, assign 0

                    else: # other index is the home team
                        home_team = team[0]
                        home_team_ranking = int(team[1]) if len(team) == 2 else 0

                find_href_results = results.find_all('div', class_='onoff') # div where box score url is
                
                # checks if url exists
                links = find_href_results[matchup_index].find_all('a', href=True)
                if(len(links) > 0):
                    for i in range(len(links)):
                        print(links[i].text)
                        if links[i].text == 'boxscore': 
                            box_score_url = links[i]['href']
                            print(links[i]['href'])
                else:
                    box_score_url = 'na'

                matchups_list.append({'away_team': away_team,
                                   'away_team_ranking': away_team_ranking,
                                   'home_team': home_team,
                                   'home_team_ranking': home_team_ranking,
                                   'neutral': neutral,
                                   'box_score_url': box_score_url,
                                   'date': date_url_list[date_url_index]['date']})
                # {'away_team': 'Purdue', 'away_team_ranking': 1, 'home_team', 'home_team': 'Michigan', 'home_team_ranking': 0, 'neutral': False}

    return matchups_list

In [10]:
def getBox(box_div):
    """ 
    Scrapes website to get box score of a team
    This will gather all major statistics of every player on that team's roster
 
    Args: 
        box_div - list<str>: list of the divs that contain the table which contains the statistics 
    Returns: 
        box - dict<dict>: dictionary of every player on the team; each player is represented by a dictionary of that player's stats 
 
    """
    box = {}
    for row in range(2, len(box_div) - 2):        
        stat_results = box_div[row].find_all("td")
        stats_dict = {}
        if len(stat_results) == 14:
            for stat in range(len(stat_results)):
                if stat == 0:
                    # checks if url exists
                    if(len(stat_results[stat]('a', href=True)) > 0):
                        player_url_id = stat_results[stat].find_all('a', href=True)[0]['href']
                        player_url_id = re.split(r'r(?=[\d])|(?<=[\d])\.', player_url_id)
                        stats_dict['player_url_id'] = player_url_id[1]
                    else:
                        stats_dict['player_url_id'] = 'na'
                    
                elif stat == 1:
                    stats_dict['min_played'] = int(stat_results[stat].text.strip().split(':')[0])

                elif stat == 2:
                    fg_data = stat_results[stat].text.strip().split('-')
                    stats_dict['fg'] = int(fg_data[0])
                    stats_dict['fg_a'] = int(fg_data[1])

                elif stat == 3:
                    three_data = stat_results[stat].text.strip().split('-')
                    stats_dict['two_point'] = int(stats_dict['fg']) - int(three_data[0])
                    stats_dict['two_point_a'] = int(stats_dict['fg_a']) - int(three_data[1])
                    stats_dict['three_point'] = int(three_data[0])
                    stats_dict['three_point_a'] = int(three_data[1])

                elif stat == 4:
                    ft_data = stat_results[stat].text.strip().split('-')
                    stats_dict['ft_a'] = int(ft_data[1])
                    stats_dict['ft'] = int(ft_data[0])

                elif stat == 5:
                    stats_dict['orb'] = int(stat_results[stat].text.strip())

                elif stat == 6:
                    stats_dict['drb'] = int(stat_results[stat].text.strip())    

                elif stat == 7:
                    stats_dict['tot_reb'] = int(stat_results[stat].text.strip())

                elif stat == 8:
                    stats_dict['a'] = int(stat_results[stat].text.strip())

                elif stat == 9:
                    stats_dict['pf'] = int(stat_results[stat].text.strip())

                elif stat == 10:
                    stats_dict['stl'] = int(stat_results[stat].text.strip())

                elif stat == 11:
                    stats_dict['to'] = int(stat_results[stat].text.strip())

                elif stat == 12:
                    stats_dict['blk'] = int(stat_results[stat].text.strip())

                elif stat == 13:
                    stats_dict['pts'] = int(stat_results[stat].text.strip())

                box[stat_results[0].text.strip()] = stats_dict
        else:
            stats_dict['min_played'] = 'nan'
    return box
    

In [11]:
def getGame(url):
    """ 
    Scrapes website to gather all available information about a game including location, score, player stats, etc.
 
    Args: 
        url - str: url of the website that contains game information and box score 
    Returns: 
        game_results - dict<>: dictionary containing all information and stats of the game 
 
    """
    html_document = getHTMLdocument(url)
    soup = BeautifulSoup(html_document, 'html.parser') # soup object
    results = soup.find('div', id='sdi-rail-content') # all of the necessary info is in this div
    
    if results is not None:
        when_where_div = results.find("div", class_="sdi-quickhits").text.strip() # this div contains all the information for time and place of game
        when_where_list = re.split('When: |Where: |Officials: |Attendance: ', when_where_div)
        when_where_list = [i for i in when_where_list if i != ""]

        game_time = re.split('\xa0|, ', when_where_list[0])[0] #Extracts the time of the game ex. '7:00 PM'

        where_list = re.split(', ', when_where_list[1])

        if len(where_list) == 3: # Most games are structured ['arena', 'city', 'state']
            arena = where_list[0]
            city = where_list[1]
            state = where_list[2]

        else: # Out of country games are strucured ['arena', 'city']
            arena = where_list[0]
            city = where_list[1]
            state = None

        if len(when_where_list) == 4: # some divs dont have attendance, ones that dont will not be len 4
            if when_where_list[3].strip().isnumeric:
                attendance = when_where_list[3].strip()
            else:
                attendance = None # some have blank attendance or non numeric
        else:
            attendance = None

        # divs that contains the scores and boxscorse of the game for both teams
        results_div = results.find_all("div", class_="sdi-so")     
        score_div = results_div[0].find_all(class_='sdi-datacell')

        # len of score_div depends on how many overtimes there were, that is what this is checking
        if len(score_div) == 8:
            away_1 = int(score_div[1].text)
            away_2 = int(score_div[2].text)
            away_ot = 0
            away_ot2 = 0
            away_ot3 = 0
            away_tot = away_1 + away_2 + away_ot + away_ot2 + away_ot3

            home_1 = int(score_div[5].text)
            home_2 = int(score_div[6].text)
            home_ot = 0
            home_ot2 = 0
            home_ot3 = 0
            home_tot = home_1 + home_2 + home_ot + home_ot2 + home_ot3

        elif len(score_div) == 10:
            away_1 = int(score_div[1].text)
            away_2 = int(score_div[2].text)
            away_ot = int(score_div[3].text)
            away_ot2 = 0
            away_ot3 = 0
            away_tot = away_1 + away_2 + away_ot + away_ot2 + away_ot3

            home_1 = int(score_div[6].text)
            home_2 = int(score_div[7].text)
            home_ot = int(score_div[8].text)
            home_ot2 = 0
            home_ot3 = 0
            home_tot = home_1 + home_2 + home_ot + home_ot2 + home_ot3

        elif len(score_div) == 12:
            away_1 = int(score_div[1].text)
            away_2 = int(score_div[2].text)
            away_ot = int(score_div[3].text)
            away_ot2 = int(score_div[4].text)
            away_ot3 = 0
            away_tot = away_1 + away_2 + away_ot + away_ot2 + away_ot3

            home_1 = int(score_div[7].text)
            home_2 = int(score_div[8].text)
            home_ot = int(score_div[9].text)
            home_ot2 = int(score_div[10].text)
            home_ot3 = 0
            home_tot = home_1 + home_2 + home_ot + home_ot2 + home_ot3

        elif len(score_div) == 14:
            away_1 = int(score_div[1].text)
            away_2 = int(score_div[2].text)
            away_ot = int(score_div[3].text)
            away_ot2 = int(score_div[4].text)
            away_ot3 = int(score_div[5].text)
            away_tot = away_1 + away_2 + away_ot + away_ot2 + away_ot3

            home_1 = int(score_div[8].text)
            home_2 = int(score_div[9].text)
            home_ot = int(score_div[10].text)
            home_ot2 = int(score_div[11].text)
            home_ot3 = int(score_div[12].text)
            home_tot = home_1 + home_2 + home_ot + home_ot2 + home_ot3


        if len(results_div) == 3:
            home_box = results_div[2].find_all("tr")
            away_box = results_div[1].find_all("tr")

            away = getBox(away_box)
            home = getBox(home_box)
        
        else:
            away = 'na'
            home = 'na'

        game_results = {'game_time': game_time,
                'arena': arena,
                'city': city,
                'state': state,
                'attendance': attendance,
                'away_1': away_1,
                'away_2': away_2,
                'away_ot': away_ot,
                'away_ot2': away_ot2,
                'away_ot3': away_ot3,
                'away_tot': away_tot,
                'home_1': home_1,
                'home_2': home_2,
                'home_ot': home_ot,
                'home_ot2': home_ot2,
                'home_ot3': home_ot3,
                'home_tot': home_tot,
                'away': away,
                'home': home}

        return game_results
    else:
        return {'game_time': 'na',
                'arena': 'na',
                'city': 'na',
                'state': 'na',
                'attendance': 'na',
                'away_1': 'na',
                'away_2': 'na',
                'away_ot': 'na',
                'away_ot2': 'na',
                'away_ot3': 'na',
                'away_tot': 'na',
                'home_1': 'na',
                'home_2': 'na',
                'home_ot': 'na',
                'home_ot2': 'na',
                'home_ot3': 'na',
                'home_tot': 'na',
                'away': 'na',
                'home': 'na'}
    
    

In [12]:
def createBoxScores(matchups, start, end):
    """ 
    Scrapes website to gather all games that were played in a given time frame
    Will gather the teams, rankings, url for the box score, and date
 
    Args: 
        date_url_list - list<dict>: dictionary containing all the urls along with the dates 
    Returns: 
        matchups_list - list<dict>: dictionary containing necessary matchup components 
 
    """
    box_scores_list = []
    
    for i in range(start, end):
        if matchups[i]['box_score_url'] != 'na':
            print(str(i - start) + '/' + str(end - start) + ' - ' + matchups[i]['away_team'] + ' vs ' + matchups[i]['home_team'] + ': ' + matchups[i]['date'])
            box_scores_list.append(getGame("https://newsday.sportsdirectinc.com" + matchups[i]['box_score_url']))  
    return box_scores_list

### Create dataframe of matchups

In [13]:
date_list = createDates(date(2010, 11, 8), date(2023, 4, 3))
date_url_list = createDatesURL(date_list)

In [14]:
# box_scores_list = createBoxScores(matchups_dict, 17398, 23389)

In [15]:
# df = pd.DataFrame(box_scores_list)
# df.to_csv('2013-2014.csv')

In [16]:
overnight = [(59139, 64163),
             (64163, 70393),
             (70393, 76627)
             ]

In [17]:
for i in range(len(overnight)):
    box_scores_list = createBoxScores(matchups_dict, overnight[i][0], overnight[i][1])
    df = pd.DataFrame(box_scores_list)
    df.to_csv(f'{str(2020 + i)}-{str(2021 + i)}.csv')

4/5024 - East Tennessee St. vs Abilene Christian: 11/25/2020
5/5024 - Limestone vs James Madison: 11/25/2020
6/5024 - Radford vs Virginia Tech: 11/25/2020
7/5024 - Oakland vs Xavier: 11/25/2020
8/5024 - North Dakota vs Miami (OH): 11/25/2020
9/5024 - Florida National vs Georgia Southern: 11/25/2020
10/5024 - Keene State vs New Hampshire: 11/25/2020
11/5024 - Lander vs Chattanooga: 11/25/2020
12/5024 - Southern Wesleyan vs South Carolina Upstate: 11/25/2020
13/5024 - Appalachian St. vs South Carolina State: 11/25/2020
14/5024 - Fairfield vs Providence: 11/25/2020
15/5024 - North Carolina A&T vs Illinois: 11/25/2020
16/5024 - Piedmont vs The Citadel: 11/25/2020
17/5024 - Toledo vs Bradley: 11/25/2020
18/5024 - Alcorn St. vs UAB: 11/25/2020
19/5024 - UNC Wilmington vs Western Carolina: 11/25/2020
20/5024 - Bowling Green vs Michigan: 11/25/2020
21/5024 - Delaware State vs Wake Forest: 11/25/2020
22/5024 - Tusculum vs Furman: 11/25/2020
23/5024 - North Carolina Central vs Iowa: 11/25/2020
2

173/5024 - Arkansas-Pine Bluff vs Wisconsin: 11/27/2020
176/5024 - Middle Tennessee St. vs East Tennessee St.: 11/27/2020
177/5024 - Virginia vs San Francisco: 11/27/2020
178/5024 - Norfolk State vs James Madison: 11/27/2020
179/5024 - Florida Atlantic vs Jacksonville St.: 11/27/2020
180/5024 - Northwest University vs Oregon St.: 11/27/2020
181/5024 - Western Kentucky vs West Virginia: 11/27/2020
182/5024 - George Mason vs Belmont: 11/27/2020
183/5024 - South Dakota vs Drake: 11/27/2020
184/5024 - Flagler College vs Florida International: 11/27/2020
185/5024 - Rice vs Incarnate Word: 11/27/2020
186/5024 - Sam Houston St. vs Texas Tech: 11/27/2020
187/5024 - Bryant vs Syracuse: 11/27/2020
188/5024 - Boise St. vs Houston: 11/27/2020
189/5024 - Seattle vs William Jessup: 11/27/2020
190/5024 - UNC Wilmington vs UNC Asheville: 11/27/2020
191/5024 - Toccoa Falls vs Georgia St.: 11/27/2020
192/5024 - La Salle vs Saint Peter's: 11/27/2020
193/5024 - Arkansas St. vs Marshall: 11/27/2020
194/502

347/5024 - South Dakota vs Nebraska: 12/1/2020
348/5024 - Kansas vs Kentucky: 12/1/2020
349/5024 - Nicholls State vs Saint Mary's: 12/1/2020
351/5024 - Southern California vs Brigham Young: 12/1/2020
352/5024 - St. Francis (PA) vs Virginia: 12/1/2020
353/5024 - Bryant vs New Hampshire: 12/1/2020
354/5024 - North Carolina Wesleyan vs East Carolina: 12/1/2020
355/5024 - Nebraska Omaha vs Creighton: 12/1/2020
356/5024 - Villanova vs Hartford: 12/1/2020
357/5024 - Stony Brook vs Saint Peter's: 12/1/2020
358/5024 - Lincoln PA vs Morgan St.: 12/1/2020
359/5024 - Oklahoma State vs Marquette: 12/1/2020
360/5024 - Emmanuel vs South Alabama: 12/1/2020
361/5024 - Davidson vs Providence: 12/1/2020
362/5024 - Central Michigan vs Florida International: 12/1/2020
363/5024 - Michigan St vs Duke: 12/1/2020
364/5024 - Hampton vs George Washington: 12/1/2020
365/5024 - Howard Payne vs Abilene Christian: 12/1/2020
366/5024 - Mississippi Valley State vs Grand Canyon: 12/1/2020
367/5024 - UNLV vs Alabama: 1

537/5024 - Coppin St. vs George Washington: 12/5/2020
538/5024 - Eastern Washington vs Arizona: 12/5/2020
539/5024 - South Carolina State vs UNC Asheville: 12/5/2020
540/5024 - Salem International vs Delaware: 12/5/2020
541/5024 - Lewis vs Loyola-Chicago: 12/5/2020
542/5024 - Lamar vs Air Force: 12/5/2020
543/5024 - Incarnate Word vs Texas State: 12/5/2020
544/5024 - North Dakota State vs Kansas: 12/5/2020
545/5024 - Iona vs Hofstra: 12/5/2020
546/5024 - Samford vs Belmont: 12/5/2020
547/5024 - Lipscomb vs Arkansas: 12/5/2020
548/5024 - Illinois St. vs Murray St.: 12/5/2020
549/5024 - Georgia Southern vs South Carolina Upstate: 12/5/2020
550/5024 - Florida National vs Florida Gulf Coast: 12/5/2020
551/5024 - Northern Illinois vs Pittsburgh: 12/5/2020
552/5024 - UNLV vs Kansas State: 12/5/2020
553/5024 - Tarleton State vs Abilene Christian: 12/5/2020
554/5024 - Arkansas-Pine Bluff vs Saint Louis: 12/5/2020
555/5024 - Portland St. vs Portland: 12/5/2020
559/5024 - Villanova vs Texas: 12/

720/5024 - Georgia Tech vs Nebraska: 12/9/2020
721/5024 - Mobile vs UAB: 12/9/2020
722/5024 - McKendree vs Drake: 12/9/2020
723/5024 - Texas State vs Texas: 12/9/2020
724/5024 - Southern Miss vs Tulane: 12/9/2020
725/5024 - Southern vs Arkansas: 12/9/2020
726/5024 - Liberty vs Missouri: 12/9/2020
727/5024 - Santa Clara vs Cal Poly SLO: 12/9/2020
728/5024 - Idaho vs Washington St.: 12/9/2020
729/5024 - California vs Pepperdine: 12/9/2020
730/5024 - Pittsburgh vs Northwestern: 12/9/2020
731/5024 - Florida A&M vs Oregon: 12/9/2020
735/5024 - Purdue Northwest vs Ohio: 12/10/2020
736/5024 - Mercyhurst vs Buffalo: 12/10/2020
737/5024 - North Florida vs East Carolina: 12/10/2020
738/5024 - North Dakota vs South Dakota: 12/10/2020
739/5024 - UNC Greensboro vs Coppin St.: 12/10/2020
740/5024 - Portland vs Oregon St.: 12/10/2020
741/5024 - UC Riverside vs Northern Arizona: 12/10/2020
742/5024 - Gardner-Webb vs Western Kentucky: 12/10/2020
743/5024 - North Dakota State vs South Dakota State: 12/1

...

5416/6234 - Towson vs Charleston: 2/23/2023
5417/6234 - Monmouth-NJ vs Hampton: 2/23/2023
5418/6234 - Northeastern vs Drexel: 2/23/2023
5419/6234 - California Baptist vs Sam Houston St.: 2/23/2023
5420/6234 - Robert Morris vs IUPUI: 2/23/2023
5421/6234 - Southeastern Louisiana vs Lamar: 2/23/2023
5422/6234 - Southern Utah vs Texas-Arlington: 2/23/2023
5423/6234 - St. Thomas (MN) vs North Dakota State: 2/23/2023
5424/6234 - Kansas City vs South Dakota State: 2/23/2023
5425/6234 - Tennessee Tech vs Southern Indiana: 2/23/2023
5426/6234 - Northwestern St. vs Incarnate Word: 2/23/2023
5427/6234 - Michigan vs Rutgers: 2/23/2023
5428/6234 - Sacramento State vs Montana St.: 2/23/2023
5429/6234 - Eastern Washington vs Weber St.: 2/23/2023
5430/6234 - Washington vs California: 2/23/2023
5431/6234 - Cleveland St. vs Green Bay: 2/23/2023
5432/6234 - Tennessee St. vs Tenn-Martin: 2/23/2023
5433/6234 - Pepperdine vs Santa Clara: 2/23/2023
5434/6234 - CSU Bakersfield vs CSU Northridge: 2/23/2023
543

5580/6234 - Chattanooga vs Wofford: 2/25/2023
5581/6234 - Portland vs Pacific: 2/25/2023
5582/6234 - Tarleton State vs Texas Rio Grande Valley: 2/25/2023
5583/6234 - DePaul vs Marquette: 2/25/2023
5584/6234 - Cleveland St. vs Milwaukee: 2/25/2023
5585/6234 - Southern California vs Utah: 2/25/2023
5586/6234 - Eastern Washington vs Idaho State: 2/25/2023
5587/6234 - Louisiana State vs Mississippi: 2/25/2023
5588/6234 - Sacramento State vs Montana: 2/25/2023
5589/6234 - UC Santa Barbara vs UC San Diego: 2/25/2023
5590/6234 - Saint Mary's vs Gonzaga: 2/25/2023
5591/6234 - San Francisco vs Brigham Young: 2/25/2023
5592/6234 - Loyola Marymount vs Pepperdine: 2/25/2023
5593/6234 - Saint Joseph's vs St. Bonaventure: 2/26/2023
5594/6234 - Northwestern vs Maryland: 2/26/2023
5595/6234 - Belmont vs Northern Iowa: 2/26/2023
5596/6234 - Siena vs Iona: 2/26/2023
5597/6234 - Cincinnati vs Memphis: 2/26/2023
5598/6234 - Missouri St. vs Indiana St.: 2/26/2023
5599/6234 - Manhattan vs Quinnipiac: 2/26/2

5748/6234 - Lafayette vs Lehigh: 3/2/2023
5749/6234 - American U. vs Navy: 3/2/2023
5750/6234 - Valparaiso vs Murray St.: 3/2/2023
5751/6234 - Lipscomb vs Kennesaw St.: 3/2/2023
5752/6234 - SE Missouri St. vs Tennessee St.: 3/2/2023
5753/6234 - Maryland-Eastern Shore vs Delaware State: 3/2/2023
5754/6234 - North Carolina Central vs South Carolina State: 3/2/2023
5755/6234 - Florida Atlantic vs Rice: 3/2/2023
5756/6234 - Middle Tennessee St. vs North Texas: 3/2/2023
5757/6234 - Wright St. vs Milwaukee: 3/2/2023
5758/6234 - Alcorn St. vs Mississippi Valley State: 3/2/2023
5759/6234 - Jackson State vs Arkansas-Pine Bluff: 3/2/2023
5760/6234 - Memphis vs Southern Methodist: 3/2/2023
5761/6234 - Stanford vs Oregon St.: 3/2/2023
5762/6234 - UC Santa Barbara vs UC Davis: 3/2/2023
5763/6234 - Norfolk State vs Howard: 3/2/2023
5764/6234 - Illinois-Chicago vs Missouri St.: 3/2/2023
5765/6234 - UC Irvine vs UC Riverside: 3/2/2023
5766/6234 - Hawaii vs CSU Northridge: 3/2/2023
5767/6234 - Washingt

5917/6234 - Michigan vs Indiana: 3/5/2023
5918/6234 - McNeese St. vs Texas A&M Commerce: 3/5/2023
5919/6234 - Chattanooga vs Wofford: 3/5/2023
5920/6234 - Northern Arizona vs Eastern Washington: 3/5/2023
5921/6234 - Wisconsin vs Minnesota: 3/5/2023
5922/6234 - Texas State vs UL Lafayette: 3/5/2023
5923/6234 - South Dakota vs North Dakota State: 3/5/2023
5924/6234 - McNeese St. vs Nicholls State: 3/6/2023
5925/6234 - St. Thomas (MN) vs Oral Roberts: 3/6/2023
5926/6234 - South Alabama vs UL Lafayette: 3/6/2023
5927/6234 - Idaho State vs Montana: 3/6/2023
5928/6234 - Towson vs Charleston: 3/6/2023
5929/6234 - Cleveland St. vs Milwaukee: 3/6/2023
5930/6234 - Sacramento State vs Weber St.: 3/6/2023
5931/6234 - UNC Wilmington vs Hofstra: 3/6/2023
5932/6234 - Chattanooga vs Furman: 3/6/2023
5933/6234 - Northern Kentucky vs Youngstown St.: 3/6/2023
5934/6234 - New Orleans vs Southeastern Louisiana: 3/6/2023
5935/6234 - Brigham Young vs Saint Mary's: 3/6/2023
5936/6234 - North Dakota State vs S

6085/6234 - Wichita St. vs Tulane: 3/10/2023
6086/6234 - Maryland vs Indiana: 3/10/2023
6087/6234 - CSU Fullerton vs UC Irvine: 3/10/2023
6088/6234 - Creighton vs Xavier: 3/10/2023
6089/6234 - Texas Christian vs Texas: 3/10/2023
6090/6234 - Jackson State vs Grambling State: 3/10/2023
6091/6234 - Arizona State vs Arizona: 3/10/2023
6092/6234 - Utah St. vs Boise St.: 3/10/2023
6093/6234 - Cornell vs Yale: 3/11/2023
6094/6234 - Saint Louis vs VCU: 3/11/2023
6095/6234 - Missouri vs Alabama: 3/11/2023
6096/6234 - Pennsylvania vs Princeton: 3/11/2023
6097/6234 - Vanderbilt vs Texas A&M: 3/11/2023
6098/6234 - Penn St. vs Indiana: 3/11/2023
6099/6234 - Tulane vs Memphis: 3/11/2023
6100/6234 - Utah St. vs San Diego St.: 3/11/2023
6101/6234 - Kent St. vs Toledo: 3/11/2023
6102/6234 - Duke vs Virginia: 3/11/2023
6103/6234 - CSU Fullerton vs UC Santa Barbara: 3/11/2023
6104/6234 - Grand Canyon vs Southern Utah: 3/11/2023
6105/6234 - Massachusetts Lowell vs Vermont: 3/11/2023
6106/6234 - Ohio St. v

### Create dataframes for 

### Create dataframe of teams

teams = pd.DataFrame(cbb_matchups['home_team'].unique(), columns=['team'])

teams

# function to create list of urls for the team pages
def createTeamURL():
    url_team_list = []

    for team in teams['lookup']:
#         for year in range(2011, 2024):
        url = "https://www.sports-reference.com/cbb/schools/" + team + "/men/2023.html"
        url_team_list.append(url)
        
    return url_team_list

# cleanup dataframe

teams['lookup'] = teams['team'].str.lower()
teams['lookup'] = teams['lookup'].str.replace(" ", "-", regex=True)
teams['lookup'] = teams['lookup'].str.replace("(", "", regex=True)
teams['lookup'] = teams['lookup'].str.replace(")", "", regex=True)
teams['lookup'] = teams['lookup'].str.replace(".", "", regex=True)
teams['lookup'] = teams['lookup'].str.replace("'", "", regex=True)
teams['lookup'] = teams['lookup'].str.replace("&", "", regex=True)
teams['lookup'] = teams['lookup'].str.replace("-st", "-state", regex=True)
teams['lookup'] = teams['lookup'].str.replace("-stateate", "-state", regex=True)
teams['lookup'] = teams['lookup'].str.replace("maryland---e-shore", "maryland-eastern-shore", regex=True)
teams['lookup'] = teams['lookup'].str.replace("tenn-martin", "tennessee-martin", regex=True)
teams['lookup'] = teams['lookup'].str.replace("nc-state", "north-carolina-state", regex=True)
teams['lookup'] = teams['lookup'].str.replace("vcu", "virginia-commonwealth", regex=True)
teams['lookup'] = teams['lookup'].str.replace("american-u", "american", regex=True)
teams['lookup'] = teams['lookup'].str.replace("middle-tennessee-state", "middle-tennessee", regex=True)
teams['lookup'] = teams['lookup'].str.replace("liu-brooklyn", "long-island-university", regex=True)
teams['lookup'] = teams['lookup'].str.replace("miami-florida", "miami-fl", regex=True)
teams['lookup'] = teams['lookup'].str.replace("uc-santa-barbara", "california-santa-barbara", regex=True)
teams['lookup'] = teams['lookup'].str.replace("st-josephs", "saint-josephs", regex=True)
teams['lookup'] = teams['lookup'].str.replace("loyola-maryland", "loyola-md", regex=True)
teams['lookup'] = teams['lookup'].str.replace("vmi", "virginia-military-institute", regex=True)
teams['lookup'] = teams['lookup'].str.replace("siu---edwardsville", "southern-illinois-edwardsville", regex=True)
teams['lookup'] = teams['lookup'].str.replace("albany", "albany-ny", regex=True)
teams['lookup'] = teams['lookup'].str.replace("loyola-chicago", "loyola-il", regex=True)
teams['lookup'] = teams['lookup'].str.replace("unlv", "nevada-las-vegas", regex=True)
teams['lookup'] = teams['lookup'].str.replace("st-marys", "saint-marys-ca", regex=True)
teams['lookup'] = teams['lookup'].str.replace("texas-am-cc", "texas-am-corpus-christi", regex=True)
teams['lookup'] = teams['lookup'].str.replace("new-jersey-tech", "njit", regex=True)
teams['lookup'] = teams['lookup'].str.replace("wis-milwaukee", "milwaukee", regex=True)
teams['lookup'] = teams['lookup'].str.replace("uab", "alabama-birmingham", regex=True)
teams['lookup'] = teams['lookup'].str.replace("md-baltimore-cty", "maryland-baltimore-county", regex=True)
teams['lookup'] = teams['lookup'].str.replace("nc-greensboro", "north-carolina-greensboro", regex=True)
teams['lookup'] = teams['lookup'].str.replace("st-francis-pa", "saint-francis-pa", regex=True)
teams['lookup'] = teams['lookup'].str.replace("uc-riverside", "california-riverside", regex=True)
teams['lookup'] = teams['lookup'].str.replace("elon-university", "elon", regex=True)
teams['lookup'] = teams['lookup'].str.replace("bowling-green", "bowling-green-state", regex=True)
teams['lookup'] = teams['lookup'].str.replace("monmouth-nj", "monmouth", regex=True)
teams['lookup'] = teams['lookup'].str.replace("nc-wilmington", "north-carolina-wilmington", regex=True)
teams['lookup'] = teams['lookup'].str.replace("charleston", "college-of-charleston", regex=True)
teams['lookup'] = teams['lookup'].str.replace("indiana---purdue", "iupui", regex=True)
teams['lookup'] = teams['lookup'].str.replace("prairie-view-am", "prairie-view", regex=True)
teams['lookup'] = teams['lookup'].str.replace("nocarolina-at", "north-carolina-at", regex=True)
teams['lookup'] = teams['lookup'].str.replace("st-johns", "st-johns-ny", regex=True)
teams['lookup'] = teams['lookup'].str.replace("cal-state---bakersfield", "cal-state-bakersfield", regex=True)
teams['lookup'] = teams['lookup'].str.replace("uc-davis", "california-davis", regex=True)
teams['lookup'] = teams['lookup'].str.replace("cal-poly-slo", "cal-poly", regex=True)
teams['lookup'] = teams['lookup'].str.replace("uc-irvine", "california-irvine", regex=True)
teams['lookup'] = teams['lookup'].str.replace("boston-u", "boston-university", regex=True)
teams['lookup'] = teams['lookup'].str.replace("nc-asheville", "north-carolina-asheville", regex=True)
teams['lookup'] = teams['lookup'].str.replace("csu-northridge", "cal-state-northridge", regex=True)
teams['lookup'] = teams['lookup'].str.replace("mount-state-marys", "mount-st-marys", regex=True)
teams['lookup'] = teams['lookup'].str.replace("grambling-state", "grambling", regex=True)
teams['lookup'] = teams['lookup'].str.replace("se-missouri-state", "southeast-missouri-state", regex=True)
teams['lookup'] = teams['lookup'].str.replace("william--mary", "william-mary", regex=True)
teams['lookup'] = teams['lookup'].str.replace("iupu---ft-wayne", "ipfw", regex=True)
teams['lookup'] = teams['lookup'].str.replace("central-conn-state", "central-connecticut-state", regex=True)
teams['lookup'] = teams['lookup'].str.replace("st-peters", "saint-peters", regex=True)
teams['lookup'] = teams['lookup'].str.replace("college-of-charleston-southern", "charleston-southern", regex=True)
teams['lookup'] = teams['lookup'].str.replace("umkc", "missouri-kansas-city", regex=True)
teams['lookup'] = teams['lookup'].str.replace("southern-miss", "southern-mississippi", regex=True)
teams['lookup'] = teams['lookup'].str.replace("the-citadel", "citadel", regex=True)
teams['lookup'] = teams['lookup'].str.replace("texas-rio-grande-valley", "texas-pan-american", regex=True)
teams['lookup'] = teams['lookup'].str.replace("little-rock", "arkansas-little-rock", regex=True)
teams['lookup'] = teams['lookup'].str.replace("lu-lafayette", "louisiana-lafayette", regex=True)
teams['lookup'] = teams['lookup'].str.replace("st-francis-brooklyn", "st-francis-ny", regex=True)
teams['lookup'] = teams['lookup'].str.replace("ul-monroe", "louisiana-monroe", regex=True)
teams['lookup'] = teams['lookup'].str.replace("unc-wilmington", "north-carolina-wilmington", regex=True)
teams['lookup'] = teams['lookup'].str.replace("unorth-carolina-asheville", "north-carolina-asheville", regex=True)
teams['lookup'] = teams['lookup'].str.replace("unorth-carolina-greensboro", "north-carolina-greensboro", regex=True)
teams['lookup'] = teams['lookup'].str.replace("unorth-carolina-wilmington", "north-carolina-wilmington", regex=True)
teams['lookup'] = teams['lookup'].str.replace("csu-fullerton", "cal-state-fullerton", regex=True)
teams['lookup'] = teams['lookup'].str.replace("houston-christian", "houston-baptist", regex=True)
teams['lookup'] = teams['lookup'].str.replace("queens-university-of-charlotte", "queens-nc", regex=True)

# find indecies of rows that need deleted
bad_team_index = []
for team in bad_teams:
    bad_team_index.append(teams.index[teams['lookup'] == team].tolist()[0])

print(bad_team_index)

# delete rows of teams that shouldn't be there
for i in bad_team_index:
    teams = teams.drop(i)
teams = teams.reset_index()

teams

url_team_list = createTeamURL()

# for url in url_team_list:
#     testURL(url)
#     time.sleep(3.1)

url_team_set = set(url_team_list)

print(len(url_team_set))

def getDivisions(url_list):
    divisions = []

    for url in url_list:
        # create document
        html_document = getHTMLdocument(url)

        # create soap object
        soup = BeautifulSoup(html_document, 'html.parser')

        results = soup.find(id="meta")
        # print(results)

        if results is not None:
            division_text = results.find_all("div")[1].find("p").text
            if " in " in division_text:
                division_text_before = division_text.index(" in ") + 4
                division_text_after = division_text[division_text_before : len(division_text)].index("MBB")
                division = division_text[division_text_before : division_text_after + division_text_before - 1]
            else:
                division = "Independent"
            
            divisions.append(division)
        
        time.sleep(3.1)
        print(url, division)
    return divisions

# divs = getDivisions(url_team_set)

# divs_set = set(divs)

# for div in divs_set:
#     print(divs.count(div), div)

https://www.sports-reference.com/cbb/schools/purdue/men/2023.html

Western Illinois summit->ovc
byu wcc->big 12
houston wcc->big 12
cincinati wcc->big 12
ucf wcc->big 12